In [1]:
import os
import operator
from num2words import num2words
import gc

In [2]:
INPUT_PATH = r'../input'
DATA_INPUT_PATH = r'../input/en_with_types'
SUBM_PATH = r'../submissions'
train_file = "en_train.csv"
test_file = "en_test.csv"
submit_file = "baseline_ext_en.csv"

# For debug purposes
#INPUT_PATH = r'./'
#DATA_INPUT_PATH = r'./en_with_types'
#SUBM_PATH = r'.'
#train_file = "en_sample_train.csv"
#test_file = "en_sample_test.csv"
#submit_file = "en_sample_submit.csv"


Training on given training set

In [3]:
train = open(os.path.join(INPUT_PATH, train_file), encoding='UTF8')

In [4]:
res = dict()
total = 0
not_same = 0

line = train.readline()
while 1:
    line = train.readline().strip()  # strips \n from the end of sentence
    if line == '':                   # reached EOF
        print('EOF')
        break
    total += 1
    pos = line.find('","')           # find the comma after which 'before' word starts
    text = line[pos + 2:]            # exclude the comma and first double quote
    if text[:3] == '","':            # there was no 'before' word
        continue
    text = text[1:-1]                # exclude the last double quote
    arr = text.split('","')          # split the 'before' and 'after' words
    
    if arr[0] != arr[1]:
        not_same += 1                # just counting

    if arr[0] not in res:            # never seen this word, create a dict for it
        res[arr[0]] = dict()         # its dict stores all the possible 'after' words
        res[arr[0]][arr[1]] = 1      # and compute frequency of the 'after' words
    else:
        if arr[1] in res[arr[0]]:
            res[arr[0]][arr[1]] += 1
        else:
            res[arr[0]][arr[1]] = 1

EOF


In [5]:
train.close()

print(train_file + ':\tTotal: {} Have diff value: {}'.format(total, not_same))

en_train.csv:	Total: 9918441 Have diff value: 659793


This code block is for the real big data

Generate the word frequency from all the available huge datasets

In [6]:
en_with_types_files = os.listdir(DATA_INPUT_PATH)

for file in en_with_types_files:
    train = open(os.path.join(DATA_INPUT_PATH, file), encoding='UTF8')
    while 1:
        line = train.readline().strip()
        if line == '':
            break
        total += 1
        pos = line.find('\t')                       # find 1st index of tab
        text = line[pos + 1:]                       # ignores class
        if text[:3] == '':
            continue
        arr = text.split('\t')
        
        if arr[0] == '<eos>':                       # Nothing to do if we reach end of sentence
            continue
        if arr[1] != '<self>':
            not_same += 1

        if arr[1] == '<self>' or arr[1] == 'sil':   # 'before' word is equal to 'after' word
            arr[1] = arr[0]                         # for PLAIN and PUNCT class words

        if arr[1] == '<self>' or arr[1] == 'sil':
            arr[1] = arr[0]

        if arr[0] not in res:
            res[arr[0]] = dict()
            res[arr[0]][arr[1]] = 1
        else:
            if arr[1] in res[arr[0]]:
                res[arr[0]][arr[1]] += 1
            else:
                res[arr[0]][arr[1]] = 1
    train.close()
    print(file + ':\tTotal: {} Have diff value: {}'.format(total, not_same))
    gc.collect()

output-00067-of-00100:	Total: 21357528 Have diff value: 3556052
output-00066-of-00100:	Total: 32789199 Have diff value: 6443038
output-00010-of-00100:	Total: 44220262 Have diff value: 9332958
output-00011-of-00100:	Total: 55701507 Have diff value: 12238835
output-00082-of-00100:	Total: 67140402 Have diff value: 15129781
output-00083-of-00100:	Total: 78594357 Have diff value: 18025115
output-00059-of-00100:	Total: 90088539 Have diff value: 20934462
output-00058-of-00100:	Total: 101532074 Have diff value: 23826666
output-00024-of-00100:	Total: 112967410 Have diff value: 26716296
output-00025-of-00100:	Total: 124392820 Have diff value: 29602913
output-00053-of-00100:	Total: 135844633 Have diff value: 32500193
output-00052-of-00100:	Total: 147230157 Have diff value: 35379195
output-00088-of-00100:	Total: 158694758 Have diff value: 38275649
output-00089-of-00100:	Total: 170115705 Have diff value: 41163164
output-00070-of-00100:	Total: 181547151 Have diff value: 44053382
output-00071-of-0010

In [7]:
res

{'Brillantaisia': {'Brillantaisia': 8},
 'is': {'i s': 1, 'is': 9200018},
 'a': {'a': 13061027},
 'genus': {'genus': 132821},
 'of': {'o f': 46, 'of': 25004172},
 'plant': {'plant': 98690},
 'in': {'in': 19384631},
 'family': {'family': 616683},
 'Acanthaceae': {'Acanthaceae': 326},
 '.': {' dot': 3, '.': 88842572},
 '2006': {'two o o six': 2114, 'two thousand six': 576352},
 'IUCN': {'i u c n': 57511},
 'Red': {'Red': 157353},
 'List': {'List': 158388},
 'Threatened': {'Threatened': 48186},
 'Species': {'Species': 107077},
 'Circa': {'Circa': 1148},
 'Survive': {'Survive': 1660},
 'draw': {'draw': 28044},
 'influences': {'influences': 16909},
 'from': {'from': 4220198},
 'soft': {'soft': 17864},
 'rock': {'rock': 103792},
 'post': {'post': 106796},
 'hardcore': {'hardcore': 5396},
 'experimental': {'experimental': 19490},
 'emo': {'emo': 577},
 'progressive': {'progressive': 12886},
 'art': {'art': 123478},
 'and': {'and': 19080614},
 'pop': {'pop': 44500, 'population': 65},
 'musicia

Create some small dictionaries

In [8]:
sdict = {}                                     # Another small dictionary for units
sdict['km2'] = 'square kilometers'
sdict['km'] = 'kilometers'
sdict['kg'] = 'kilograms'
sdict['lb'] = 'pounds'
sdict['dr'] = 'doctor'
sdict['m²'] = 'square meters'

In [9]:
SUB = str.maketrans("₀₁₂₃₄₅₆₇₈₉", "0123456789") # To convert numbers to words
SUP = str.maketrans("⁰¹²³⁴⁵⁶⁷⁸⁹", "0123456789")
OTH = str.maketrans("፬", "4")

Let's begin prediction on test data

In [10]:
out = open(os.path.join(SUBM_PATH, submit_file), "w", encoding='UTF8')
test = open(os.path.join(INPUT_PATH, test_file), encoding='UTF8')

out.write('"id","after"\n')

13

In [11]:
total = 0
changes = 0
line = test.readline().strip()

while 1:
    line = test.readline().strip()
    if line == '':
        break

    pos = line.find(',')
    i1 = line[:pos]                            # get sentence id
    line = line[pos + 1:]

    pos = line.find(',')
    i2 = line[:pos]                            # get token id
    line = line[pos + 1:]

    line = line[1:-1]                          # get 'before' word
    
    out.write('"' + i1 + '_' + i2 + '",')      # write as sentence_token
    if line in res:                            # if word exits in dictionary
        srtd = sorted(res[line].items(), key=operator.itemgetter(1), reverse=True)
        out.write('"' + srtd[0][0] + '"')      # write the 'after' word with highest frequency
        changes += 1
    else:
        if len(line) > 1:
            val = line.split(',')
            if len(val) == 2 and val[0].isdigit and val[1].isdigit:
                line = ''.join(val)            # join numbers like 2,700 ft to 2700 ft

        if line.isdigit():                                # if 'before' word is a number
            srtd = line.translate(SUB)
            srtd = srtd.translate(SUP)
            srtd = srtd.translate(OTH)
            out.write('"' + num2words(float(srtd)) + '"') # direct translation of numbers to words
            changes += 1
        elif len(line.split(' ')) > 1:                    # for more than one 'before' words
            val = line.split(' ')
            for i, v in enumerate(val):
                if v.isdigit():                           # for every word
                    srtd = v.translate(SUB)               # translate it to words if it is number
                    srtd = srtd.translate(SUP)
                    srtd = srtd.translate(OTH)
                    val[i] = num2words(float(srtd))
                elif v in sdict:                          # or if it is some units, then convert using sdict
                    val[i] = sdict[v]
            out.write('"' + ' '.join(val) + '"')          # or just keep it as it is
            changes += 1
        else:                                  # single word non number 'before' word? keep it as it is
            out.write('"' + line + '"')

    out.write('\n')
    total += 1

In [12]:
test.close()
out.close()

print('Total: {} Changed: {}'.format(total, changes))

Total: 1088564 Changed: 1088560


Exploring generated submit file

In [13]:
tst = open(os.path.join(INPUT_PATH, test_file), encoding='UTF8')
outt = open(os.path.join(SUBM_PATH, submit_file), encoding='UTF8')

In [14]:
entries = 30
for i in range(entries + 1):
    print(tst.readline().strip(), "\tbecomes\t", outt.readline())

"sentence_id","token_id","before" 	becomes	 "id","after"

0,0,"Another" 	becomes	 "0_0","Another"

0,1,"religious" 	becomes	 "0_1","religious"

0,2,"family" 	becomes	 "0_2","family"

0,3,"is" 	becomes	 "0_3","is"

0,4,"of" 	becomes	 "0_4","of"

0,5,"Hazrat" 	becomes	 "0_5","Hazrat"

0,6,"Sayyed" 	becomes	 "0_6","Sayyed"

0,7,"Ahmad" 	becomes	 "0_7","Ahmad"

0,8,"and" 	becomes	 "0_8","and"

0,9,"his" 	becomes	 "0_9","his"

0,10,"nephew" 	becomes	 "0_10","nephew"

0,11,"Hazrat" 	becomes	 "0_11","Hazrat"

0,12,"Abdul" 	becomes	 "0_12","Abdul"

0,13,"Haq" 	becomes	 "0_13","Haq"

0,14,"." 	becomes	 "0_14","."

1,0,"The" 	becomes	 "1_0","The"

1,1,"free" 	becomes	 "1_1","free"

1,2,"webapp" 	becomes	 "1_2","webapp"

1,3,"functionality" 	becomes	 "1_3","functionality"

1,4,"contains" 	becomes	 "1_4","contains"

1,5,"advertisements" 	becomes	 "1_5","advertisements"

1,6,"geared" 	becomes	 "1_6","geared"

1,7,"to" 	becomes	 "1_7","to"

1,8,"small" 	becomes	 "1_8","small"

1,9,"businesses" 	beco

In [15]:
test.close()
out.close()